### Reading in data and assessing what sort of information we can get from it

In [1]:
# !pip install sodapy
# !pip install geopandas
# !pip install folium
# !pip install mapclassify

In [2]:
import warnings
warnings.filterwarnings('ignore') 

### Imports

In [613]:
# For importing the data and using API
from sodapy import Socrata
from urllib.request import urlopen
import os
import zipfile as zf
import requests
from io import BytesIO 

# Working with the data
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping, Point
#from sklearn.preprocessing import LabelEncoder
import json

# Creating Visualisations
# import plotly.graph_objs as go
# import plotly.express as px
# Visualisation
import matplotlib.pylab as plt
import folium
from IPython.core.display import display, HTML
from folium import plugins
from folium import GeoJson
import mapclassify
import seaborn as sns
import branca.colormap as cm

#### COM Population Dataset
https://data.melbourne.vic.gov.au/resource/sp4r-xphj.json

In [4]:
domain = "data.melbourne.vic.gov.au"
data_file = 'sp4r-xphj'

In [5]:
apptoken = os.environ.get("SODAPY_APPTOKEN") # Anonymous app token
client = Socrata(domain, apptoken) 

##### View the Population dataset head sample

In [6]:
population_data = pd.DataFrame.from_dict(client.get_all(data_file))
print(population_data.shape)
population_data.head()

(16989, 5)


,geography,year,gender,age,value
0,City of Melbourne,2020,Female,Age 0-4,2683
1,City of Melbourne,2021,Female,Age 0-4,2945
2,City of Melbourne,2022,Female,Age 0-4,3212
3,City of Melbourne,2023,Female,Age 0-4,3515
4,City of Melbourne,2024,Female,Age 0-4,3833


In [7]:
def pop_data_by_year(dataset, year):
    """
    Filters and cleans the Population dataset returning a new pandas dataframe focused on the year passed to the function.
    
    Note that the year must be between 2020 and 2040 inclusive. 
    """
    # Extract the colomns of interest into "summary".
    summary = dataset[['geography', 'year', 'value']]
    # Convert datatypes and rename geography to suburb
    summary = summary.astype({'year':int, 'value':float, 'geography':str})
    summary.rename(columns={'geography':'suburb'}, inplace=True)
    # Extract the data matching the year passed from the summary.
    data = summary[summary['year'] == year]
   
    # Grouping the data by suburb while summing the population values. 
    data = pd.DataFrame(data.groupby('suburb')['value'].sum())
    data = data.reset_index()
    # Renaming the column "value" to "population_year" where year represents the year passed.
    data.rename(columns={'value':f'pop_{year}'}, inplace=True)
    
    # Cleaning the data and reset indexes
    data['suburb'] = data['suburb'].replace(['Melbourne (CBD)', 'Melbourne (Remainder)'], ['Melbourne', 'Melbourne'])
    data = pd.DataFrame(data.groupby('suburb')[f'pop_{year}'].sum())
    data = data.reset_index()
    
    # Removing unrequired data.
    subs_to_delete = ['West Melbourne (Industrial)', 'City of Melbourne']
    subs = [data.index[data['suburb']==sub].tolist()[0] for sub in subs_to_delete]
    data.drop(subs, inplace = True)

    data = data.reset_index(drop=True)
    data['suburb'] = data['suburb'].replace(['West Melbourne (Residential)'], ['West Melbourne'])
    
    # sort data
    data.sort_values('suburb', inplace = True)
    data = data.reset_index(drop=True)
    data['suburb'] = data['suburb'].astype(str)
    
    return data

#### Create dictionary for population data

In [8]:
def create_population_dict(data):
    '''Create dictionary with a datframe for all years in the dataset'''
    population = {}
    years = data['year'].unique().tolist()
    
    for year in years:
        population[int(year)] = year_data = pop_data_by_year(data, int(year))
    
    return population
    

population_layers = create_population_dict(population_data)
population_layers

{2020:              suburb   pop_2020
 0           Carlton   68920.81
 1         Docklands   48213.64
 2    East Melbourne   18713.34
 3        Kensington   33948.22
 4         Melbourne  175344.97
 5   North Melbourne   52697.81
 6         Parkville   31881.88
 7    Port Melbourne     141.77
 8       South Yarra   14261.88
 9         Southbank   83619.16
 10   West Melbourne   24884.36,
 2021:              suburb   pop_2021
 0           Carlton   66981.24
 1         Docklands   47711.07
 2    East Melbourne   18472.78
 3        Kensington   33988.86
 4         Melbourne  177233.63
 5   North Melbourne   51508.42
 6         Parkville   30837.88
 7    Port Melbourne     144.37
 8       South Yarra   14085.97
 9         Southbank   83582.46
 10   West Melbourne   25026.38,
 2022:              suburb   pop_2022
 0           Carlton   68646.64
 1         Docklands   49629.55
 2    East Melbourne   19080.77
 3        Kensington   34963.64
 4         Melbourne  182472.75
 5   North Melbourne

In [9]:
# Importing victorian geo data from json file.
url = (
    'https://data.gov.au/geoserver/vic-suburb-locality-boundaries-psma-administrative-boundaries/'
    + 'wfs?request=GetFeature&typeName=ckan_af33dd8c_0534_4e18_9245_fc64440f742e&outputFormat=json')
vic_suburbs = gpd.read_file(url)
vic_suburbs_reduced = vic_suburbs[['vic_loca_2', 'geometry']]
vic_suburbs_reduced.rename(columns={'vic_loca_2':'suburb'}, inplace=True)
vic_suburbs_reduced.head()

,suburb,geometry
0,UNDERBOOL,"MULTIPOLYGON (((141.74552 -35.07229, 141.74552..."
1,NURRAN,"MULTIPOLYGON (((148.66877 -37.39571, 148.66876..."
2,WOORNDOO,"MULTIPOLYGON (((142.92288 -37.97886, 142.90449..."
3,DEPTFORD,"MULTIPOLYGON (((147.82336 -37.66001, 147.82313..."
4,YANAC,"MULTIPOLYGON (((141.27978 -35.99859, 141.27989..."


In [10]:
# Extract the suburbs of interest that match the population_data into "target_suburbs".
target_suburbs = population_layers[2022]['suburb'].str.upper()

# Locate the index of the target suburbs and store as a list in "subs"
subs = [vic_suburbs_reduced.index[vic_suburbs_reduced['suburb']==sub].tolist()[0] for sub in target_suburbs]

# Create a new dataframe for the melbourne suburbs "mel_suburbs" and reformat.
mel_suburbs = pd.DataFrame(vic_suburbs_reduced.iloc[subs])
mel_suburbs = mel_suburbs.reset_index(drop=True)
mel_suburbs['suburb'] = mel_suburbs['suburb'].str.title()
mel_suburbs

,suburb,geometry
0,Carlton,"MULTIPOLYGON (((144.97401 -37.80311, 144.97320..."
1,Docklands,"MULTIPOLYGON (((144.95376 -37.82363, 144.95336..."
2,East Melbourne,"MULTIPOLYGON (((144.97136 -37.80773, 144.97308..."
3,Kensington,"MULTIPOLYGON (((144.92282 -37.79913, 144.91977..."
4,Melbourne,"MULTIPOLYGON (((144.97797 -37.83867, 144.97803..."
5,North Melbourne,"MULTIPOLYGON (((144.95599 -37.80588, 144.95360..."
6,Parkville,"MULTIPOLYGON (((144.96521 -37.79315, 144.96460..."
7,Port Melbourne,"MULTIPOLYGON (((144.90749 -37.84326, 144.90652..."
8,South Yarra,"MULTIPOLYGON (((145.00455 -37.84131, 145.00453..."
9,Southbank,"MULTIPOLYGON (((144.97041 -37.83016, 144.97030..."


In [11]:
# Convert the DF to a GeoDF
melbourne_geo_data = gpd.GeoDataFrame(mel_suburbs) 

# Convert the GeoDF to a JSON format. 
melbourne_geo_data_json = melbourne_geo_data.to_json()
melbourne_geo_data_json[:200] # 2000 Char preview

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"suburb": "Carlton"}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[144.97400757, -37.80311047], [144'

In [12]:
# Testing inbuilt GeoPands Features 

#melbourne_geo_data.explore(melbourne_geo_data.area , legend=False)
# Hover over the suburbs Anj!! I think we add this as an interactive explore our traget suburbs, its just cool.

### Function to create layers for each year

In [13]:
# Create base map with Folium

melb_map = folium.Map(
    location=[-37.81368709240999, 144.95738102347036],
    #width=500, height=300,
    tiles='Cartodb Positron',
    zoom_start=12,
    min_zoom=10)

In [14]:
def create_map_layer(dataset, dataset_name, data_legend, col_name):
    folium.Choropleth(
        geo_data = melbourne_geo_data_json,
        name = dataset_name,
        data = dataset,
        columns = ['suburb', col_name],
        key_on='feature.properties.suburb', 
        fill_color = 'YlGn',
        fill_opacity = 0.7,
        line_opacity = 0.2,
        legend_name = data_legend,
    ).add_to(melb_map)

    folium.LayerControl().add_to(melb_map)
    
    '''The tooltips are not working...'''
#     # Add tooltips
#     folium.features.GeoJson(
#     data=dataset,
#     name=dataset_name,
#     smooth_factor=2,
#     style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['suburb',
#                 col_name
#                ],
#         aliases=["Suburb:",
#                  "Population:"
#                 ], 
#         localize=True,
#         sticky=False,
#         labels=True,
#         style="""
#             background-color: #F0EFEF;
#             border: 2px solid black;
#             border-radius: 3px;
#             box-shadow: 3px;
#         """,
#         max_width=800,),
#             highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
#         ).add_to(melb_map)  


In [15]:
def create_map_feature_group(dataset, dataset_name, data_legend, col_name, data_map):
    """Almost works..."""
    fg = folium.FeatureGroup(name=dataset_name,overlay=False).add_to(data_map)
    population =folium.Choropleth(
            geo_data = melbourne_geo_data_json,
            data = dataset,
            columns = ['suburb', col_name],  
            key_on = 'feature.properties.suburb', 
#             threshold_scale=custom_scale1, #use the custom scale we created for legend
            fill_color='YlGn',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name = data_legend,
            highlight=True,
            overlay=False,
            line_color='black').geojson.add_to(fg)
    folium.LayerControl().add_to(data_map)
    return data_map

In [16]:
# Create feature groups
# fg1 = folium.FeatureGroup(name='New Covid-19 Cases Past 7 Days',overlay=False).add_to(us_map)
# fg2 = folium.FeatureGroup(name='Percent of Positive Cases Past 7 Days',overlay=False).add_to(us_map)


In [17]:
def add_population_to_map(dataframe,label, main_data, melb_map):    
    col_name = f'pop_{label}'
    
    if col_name not in main_data.columns:
        main_data = main_data.join(dataframe.set_index(["suburb"]), on=["suburb"])
    
    '''Swap these two lines to either have one layer or attempt to group layers... (one so far)'''
#     melb_map = create_map_feature_group(main_data, f'Population Density ({label})', f'Population Density',col_name, melb_map)
    create_map_layer(main_data, f'Population Density ({label})', f'Population Density',col_name)

    return main_data, melb_map

In [18]:
year = 2022
data = population_layers[year]

melbourne_geo_data, melb_map = add_population_to_map(data, year, melbourne_geo_data, melb_map)

In [19]:
# folium.TileLayer('cartodbdark_matter',overlay=True,name="View in Dark Mode").add_to(melb_map)
# folium.TileLayer('cartodbpositron',overlay=True,name="Viw in Light Mode").add_to(melb_map)
# folium.LayerControl(collapsed=False).add_to(melb_map)
# melb_map.save("index.html") 

In [20]:
#melb_map 

In [21]:
# year = 2021
# data = population_layers[year]

# add_population_to_map(data, year)

In [22]:
pop_data_2030 = pop_data_by_year(population_data, 2030)
pop_data_2030 

,suburb,pop_2030
0,Carlton,86749.87
1,Docklands,69207.61
2,East Melbourne,23736.45
3,Kensington,49074.21
4,Melbourne,249852.84
5,North Melbourne,83938.89
6,Parkville,39775.58
7,Port Melbourne,4842.04
8,South Yarra,17033.03
9,Southbank,122042.52


In [23]:
# Different years
pop_data_22 = pop_data_by_year(population_data, 2022)

In [24]:
pop_data_22.head()

,suburb,pop_2022
0,Carlton,68646.64
1,Docklands,49629.55
2,East Melbourne,19080.77
3,Kensington,34963.64
4,Melbourne,182472.75


In [25]:
# folium.Choropleth(
#     geo_data = melbourne_geo_data_json,
#     name = 'pop_22',
#     data = pop_data_22,
#     columns = ['geography', 'population'],
#     key_on='feature.properties.suburb', 
#     fill_color = 'YlGn',
#     fill_opacity = 0.7,
#     line_opacity = 0.2,
#     legend_name = 'Population Density 2022',
# ).add_to(melb_map)

# folium.LayerControl().add_to(melb_map)
# melb_map

In [26]:
pop_data_21 = pop_data_by_year(population_data, 2021)

In [27]:
# folium.Choropleth(
#     geo_data = melbourne_geo_data_json,
#     name = 'pop_21',
#     data = pop_data_21,
#     columns = ['geography', 'population'],
#     key_on='feature.properties.suburb', 
#     fill_color = 'YlGn',
#     fill_opacity = 0.7,
#     line_opacity = 0.2,
#     legend_name = 'Population Density 2021',
# ).add_to(melb_map)

# folium.LayerControl().add_to(melb_map)
# melb_map

-----

In [28]:
#returns a new GeoPandas DataFrame with the difference between the two years as a number and a percentage.
def population_diff(population_data, year_1, year_2):
    
    start_year = pop_data_by_year(population_data, year_1)
    end_year = pop_data_by_year(population_data, year_2)

    combined = start_year.merge(end_year, left_on='suburb', right_on='suburb')
    
    combined['change #'] = combined[list(end_year)[1]] - combined[list(start_year)[1]]
    combined['change %'] = combined[list(start_year)[1]] / combined[list(end_year)[1]]
    combined['geometry'] = mel_suburbs['geometry']
    
    return gpd.GeoDataFrame(combined)

In [29]:
pop_diff_2020_2030 = population_diff(population_data, 2020, 2030)
print(type(pop_diff_2020_2030))
print(pop_diff_2020_2030.dtypes)
pop_diff_2020_2030 

<class 'geopandas.geodataframe.GeoDataFrame'>
suburb        object
pop_2020     float64
pop_2030     float64
change #     float64
change %     float64
geometry    geometry
dtype: object


,suburb,pop_2020,pop_2030,change #,change %,geometry
0,Carlton,68920.81,86749.87,17829.06,0.794477,"MULTIPOLYGON (((144.97401 -37.80311, 144.97320..."
1,Docklands,48213.64,69207.61,20993.97,0.696652,"MULTIPOLYGON (((144.95376 -37.82363, 144.95336..."
2,East Melbourne,18713.34,23736.45,5023.11,0.788380,"MULTIPOLYGON (((144.97136 -37.80773, 144.97308..."
3,Kensington,33948.22,49074.21,15125.99,0.691773,"MULTIPOLYGON (((144.92282 -37.79913, 144.91977..."
4,Melbourne,175344.97,249852.84,74507.87,0.701793,"MULTIPOLYGON (((144.97797 -37.83867, 144.97803..."
5,North Melbourne,52697.81,83938.89,31241.08,0.627812,"MULTIPOLYGON (((144.95599 -37.80588, 144.95360..."
6,Parkville,31881.88,39775.58,7893.70,0.801544,"MULTIPOLYGON (((144.96521 -37.79315, 144.96460..."
7,Port Melbourne,141.77,4842.04,4700.27,0.029279,"MULTIPOLYGON (((144.90749 -37.84326, 144.90652..."
8,South Yarra,14261.88,17033.03,2771.15,0.837307,"MULTIPOLYGON (((145.00455 -37.84131, 145.00453..."
9,Southbank,83619.16,122042.52,38423.36,0.685164,"MULTIPOLYGON (((144.97041 -37.83016, 144.97030..."


In [30]:
# Simple function to return a fresh map.

def change_map_folium(population_data, layer_name, zoom):
    # Create Bse Layer 
    my_map = folium.Map(
        location=[-37.81368709240999, 144.95738102347036],
        tiles='Cartodb Positron',
        zoom_start=zoom,
        min_zoom=10)

    # Create choropleth overlay
    folium.Choropleth(
        geo_data = melbourne_geo_data_json,
        name = layer_name,
        data = population_data,
        columns = ['suburb', 'change %'], # list(population_data)[1]
        key_on='feature.properties.suburb', 
        fill_color = 'YlGn',
        fill_opacity = 0.7,
        line_opacity = 0.2,
        legend_name = f'Growth of Population {layer_name} (%)',
    ).add_to(my_map)
    
    folium.LayerControl().add_to(my_map)
    
    return my_map

In [31]:
def change_map(population_data, colour, title = "Melbourne City"): # We can remove the colour parameter, I just added it for testing colours 
    # Create a GeoPands overlay # COLOUR OPTIONS: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    geo_layer = population_data.explore(column ='change %', tiles='CartoDB positron', cmap=colour, name = title)
    folium.LayerControl().add_to(geo_layer)
    
    return geo_layer 

In [32]:
pop_change_1 = population_diff(population_data, 2020, 2025)  # Enter the two years to compare
map_2020_2025 = change_map(pop_change_1, 'winter') 

pop_change_2 = population_diff(population_data, 2025, 2030) 
map_2025_2030 = change_map(pop_change_2, 'winter')

pop_change_3 = population_diff(population_data, 2030, 2035) 
map_2030_2035 = change_map(pop_change_3, 'winter')

pop_change_4 = population_diff(population_data, 2035, 2040) 
map_2035_2040 = change_map(pop_change_4, 'winter')

pop_change_5 = population_diff(population_data, 2030, 2040) 
map_2020_2040 = change_map(pop_change_5, 'winter', 'Population Difference (2020 - 2040)')

In [33]:
def map_compare(map_1, map_2):
        htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 49%; margin: 0 auto; border: 2px solid #0f9295"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 49%; margin: 0 auto; border: 2px solid #0f9295"></iframe>'
           .format(map_1.get_root().render().replace('"', '&quot;'),400,400,
                   map_2.get_root().render().replace('"', '&quot;'),400,400))
        
        return display(htmlmap)

growth_5y_10y = map_compare(map_2020_2025, map_2025_2030)
growth_15y_20y = map_compare(map_2030_2035, map_2035_2040)

growth_5y_10y
growth_15y_20y

Insert write up about observations. 

In [34]:
#map_2020_2040

## GeoPandas Take 2

In [35]:
all_subs = gpd.read_file(url)
all_subs.head(2)

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry
0,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"MULTIPOLYGON (((141.74552 -35.07229, 141.74552..."
1,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"MULTIPOLYGON (((148.66877 -37.39571, 148.66876..."


In [36]:
all_subs_df = all_subs[['vic_loca_2', 'geometry']]
all_subs_df.rename(columns={'vic_loca_2':'suburb'}, inplace=True)
all_subs_df.head()

,suburb,geometry
0,UNDERBOOL,"MULTIPOLYGON (((141.74552 -35.07229, 141.74552..."
1,NURRAN,"MULTIPOLYGON (((148.66877 -37.39571, 148.66876..."
2,WOORNDOO,"MULTIPOLYGON (((142.92288 -37.97886, 142.90449..."
3,DEPTFORD,"MULTIPOLYGON (((147.82336 -37.66001, 147.82313..."
4,YANAC,"MULTIPOLYGON (((141.27978 -35.99859, 141.27989..."


In [37]:
# Extract the suburbs of interest that match the population_data into "target_suburbs".
target_subs = population_layers[2022]['suburb'].str.upper()

# Locate the index of the target suburbs and store as a list in "subs"
subs = [all_subs_df.index[all_subs_df['suburb']==sub].tolist()[0] for sub in target_subs]

#Remove unwanted rows and keep data in geo dataframe format
city_suburbs = all_subs_df.take(list(subs))
city_suburbs.reset_index(drop=True, inplace = True)
city_suburbs['suburb'] = city_suburbs['suburb'].str.title()

city_suburbs

,suburb,geometry
0,Carlton,"MULTIPOLYGON (((144.97401 -37.80311, 144.97320..."
1,Docklands,"MULTIPOLYGON (((144.95376 -37.82363, 144.95336..."
2,East Melbourne,"MULTIPOLYGON (((144.97136 -37.80773, 144.97308..."
3,Kensington,"MULTIPOLYGON (((144.92282 -37.79913, 144.91977..."
4,Melbourne,"MULTIPOLYGON (((144.97797 -37.83867, 144.97803..."
5,North Melbourne,"MULTIPOLYGON (((144.95599 -37.80588, 144.95360..."
6,Parkville,"MULTIPOLYGON (((144.96521 -37.79315, 144.96460..."
7,Port Melbourne,"MULTIPOLYGON (((144.90749 -37.84326, 144.90652..."
8,South Yarra,"MULTIPOLYGON (((145.00455 -37.84131, 145.00453..."
9,Southbank,"MULTIPOLYGON (((144.97041 -37.83016, 144.97030..."


In [38]:
type(city_suburbs)

geopandas.geodataframe.GeoDataFrame

In [39]:
#city_suburbs.explore()

In [40]:
city_suburbs_pop = city_suburbs.join(population_layers[2022].set_index(["suburb"]), on=["suburb"])
print(type(city_suburbs_pop))
city_suburbs_pop

<class 'geopandas.geodataframe.GeoDataFrame'>


,suburb,geometry,pop_2022
0,Carlton,"MULTIPOLYGON (((144.97401 -37.80311, 144.97320...",68646.64
1,Docklands,"MULTIPOLYGON (((144.95376 -37.82363, 144.95336...",49629.55
2,East Melbourne,"MULTIPOLYGON (((144.97136 -37.80773, 144.97308...",19080.77
3,Kensington,"MULTIPOLYGON (((144.92282 -37.79913, 144.91977...",34963.64
4,Melbourne,"MULTIPOLYGON (((144.97797 -37.83867, 144.97803...",182472.75
5,North Melbourne,"MULTIPOLYGON (((144.95599 -37.80588, 144.95360...",52874.80
6,Parkville,"MULTIPOLYGON (((144.96521 -37.79315, 144.96460...",31820.68
7,Port Melbourne,"MULTIPOLYGON (((144.90749 -37.84326, 144.90652...",144.74
8,South Yarra,"MULTIPOLYGON (((145.00455 -37.84131, 145.00453...",14510.35
9,Southbank,"MULTIPOLYGON (((144.97041 -37.83016, 144.97030...",91839.18


In [41]:
type(city_suburbs_pop)

geopandas.geodataframe.GeoDataFrame

In [42]:
#city_suburbs_pop.explore(column = 'pop_2022', name = 'Population (thousands) 2022')

In [43]:
# Save the layer view as a map layer
pop_m = city_suburbs_pop.explore(column = 'pop_2022', name = 'Population (thousands) 2022')

# Add layer control for user
folium.LayerControl().add_to(pop_m)

# display the map
pop_m

In [44]:
city_suburbs_pop = city_suburbs_pop.join(population_layers[2030].set_index(["suburb"]), on=["suburb"])
city_suburbs_pop

,suburb,geometry,pop_2022,pop_2030
0,Carlton,"MULTIPOLYGON (((144.97401 -37.80311, 144.97320...",68646.64,86749.87
1,Docklands,"MULTIPOLYGON (((144.95376 -37.82363, 144.95336...",49629.55,69207.61
2,East Melbourne,"MULTIPOLYGON (((144.97136 -37.80773, 144.97308...",19080.77,23736.45
3,Kensington,"MULTIPOLYGON (((144.92282 -37.79913, 144.91977...",34963.64,49074.21
4,Melbourne,"MULTIPOLYGON (((144.97797 -37.83867, 144.97803...",182472.75,249852.84
5,North Melbourne,"MULTIPOLYGON (((144.95599 -37.80588, 144.95360...",52874.80,83938.89
6,Parkville,"MULTIPOLYGON (((144.96521 -37.79315, 144.96460...",31820.68,39775.58
7,Port Melbourne,"MULTIPOLYGON (((144.90749 -37.84326, 144.90652...",144.74,4842.04
8,South Yarra,"MULTIPOLYGON (((145.00455 -37.84131, 145.00453...",14510.35,17033.03
9,Southbank,"MULTIPOLYGON (((144.97041 -37.83016, 144.97030...",91839.18,122042.52


In [45]:
com = city_suburbs.explore(name = 'Melbourne Cities')

city_suburbs.join(population_layers[2022].set_index(["suburb"]), on=["suburb"]).explore(m = com, column = 'pop_2022', name = 'Population (thousands) 2022')
city_suburbs.join(population_layers[2030].set_index(["suburb"]), on=["suburb"]).explore(m = com, column = 'pop_2030', name = 'Population (thousands) 2030')
city_suburbs.join(population_layers[2040].set_index(["suburb"]), on=["suburb"]).explore(m = com, column = 'pop_2040', name = 'Population (thousands) 2040')

folium.TileLayer(control=True).add_to(com)  # use folium to add alternative tiles
folium.LayerControl().add_to(com)
com

#### VicRoads Traffic Dataset

In [46]:
traffic_url = 'https://vicroadsopendata-vicroadsmaps.opendata.arcgis.com/datasets/5512df2ff41e4941bacf868053dbfba9_0.csv?outSR=%7B%22latestWkid%22%3A3111%2C%22wkid%22%3A102171%7D'

In [47]:
traffic_data = pd.read_csv(traffic_url)

In [48]:
traffic_data.head()

,OBJECTID_1,OBJECTID,TIS_ID,HMGNS_FLOW_ID,HMGNS_LNK_ID,HMGNS_LNK_DESC,LGA_SHORT_NM,RGN_LONG_NM,ROAD_NBR,DECLARED_ROAD_NM,...,TWO_WAY_AADT_TRUCKS,ALLVEH_AMPEAK_AADT,ALLVEH_PMPEAK_AADT,GROWTH_RATE,CI,AM_PEAK_SPEED,OFF_PEAK_SPEED,PM_PEAK_SPEED,YR,LABEL
0,1,743,14915,14915,2006,MARYSVILLE-WOODS POINT ROAD btwn LAKE MOUNTAI...,YARRA RANGES,METROPOLITAN SOUTH EAST REGION,4961,MARYSVILLE-WOODS POINT ROAD,...,0.0,NaN,NaN,0.013,0.005,NaN,NaN,NaN,2020,24* (13% 3*) EAST BOUND
1,2,650,14140,14140,8786,STEELS CREEK ROAD btwn WILLOWBEND DRIVE & ELT...,YARRA RANGES,METROPOLITAN SOUTH EAST REGION,9999,Not Applicable,...,40.0,NaN,NaN,0.019,0.002,NaN,NaN,NaN,2020,373* (6% 22*) NORTH BOUND
2,3,701,12113,12113,6035,LATROBE ROAD btwn TANJIL EAST ROAD & GORDON S...,LATROBE,EASTERN REGION,5911,MORWELL-YALLOURN NORTH ROAD,...,160.0,NaN,NaN,0.015,0.009,NaN,NaN,NaN,2020,"1,100* (6% 61*) NORTH BOUND"
3,4,702,12897,12897,7079,CASTERTON ROAD btwn GLENELG HIGHWAY & COLERAI...,SOUTHERN GRAMPIANS,SOUTH WESTERN REGION,2670,GLENELG HIGHWAY,...,340.0,NaN,NaN,0.020,0.001,NaN,NaN,NaN,2020,801* (21% 165*) WEST BOUND
4,5,703,9893,9893,3475,HUTTON ROAD btwn CHAPEL ROAD & GREENS ROAD,DANDENONG,METROPOLITAN SOUTH EAST REGION,5168,BRAESIDE-DANDENONG ROAD,...,1500.0,1000.0,1100.0,0.003,0.002,NaN,NaN,NaN,2020,"12,000 (6% 744*) WEST BOUND"


In [49]:
print(traffic_data['LGA_SHORT_NM'].unique())

['YARRA RANGES' 'LATROBE' 'SOUTHERN GRAMPIANS' 'DANDENONG' 'BRIMBANK'
 'WELLINGTON' 'WYNDHAM' 'KNOX' 'HUME' 'EAST GIPPSLAND' 'BAYSIDE' 'BENALLA'
 'MOIRA' 'NILLUMBIK' 'CASEY' 'LODDON' 'MAROONDAH' 'KINGSTON' 'GLENELG'
 'MORNINGTON PENINSULA' 'MANNINGHAM' 'DAREBIN' 'CORANGAMITE' 'SURF COAST'
 'TOWONG' 'WEST WIMMERA' 'BENDIGO' 'GEELONG' 'BASS COAST'
 '(MOUNT STIRLING)' 'MOYNE' 'QUEENSCLIFFE' 'BAW BAW' 'BALLARAT'
 'CENTRAL GOLDFIELDS' 'FRANKSTON' 'PYRENEES' 'WHITTLESEA' 'MARIBYRNONG'
 'INDIGO' 'MOUNT ALEXANDER' 'MELBOURNE' 'BANYULE' 'SOUTH GIPPSLAND'
 'STONNINGTON' 'GOLDEN PLAINS' 'SHEPPARTON' 'MORELAND' 'BOROONDARA'
 'NORTHERN GRAMPIANS' 'YARRA' 'ALPINE' 'WHITEHORSE' 'COLAC OTWAY'
 'WANGARATTA' 'MACEDON RANGES' 'ARARAT' 'HORSHAM' 'WODONGA' 'HOBSONS BAY'
 'CARDINIA' 'MILDURA' 'PORT PHILLIP' 'MOORABOOL' 'BULOKE' 'MITCHELL'
 'MURRINDINDI' 'MONASH' 'HINDMARSH' 'MELTON' 'MOONEE VALLEY' 'WARRNAMBOOL'
 'YARRIAMBIACK' 'MANSFIELD' 'GLEN EIRA' 'GANNAWARRA' 'CAMPASPE'
 'SWAN HILL' 'STRATHBOGIE' 'HEPB

In [50]:
melb_traffic = traffic_data.where(traffic_data['LGA_SHORT_NM']=='MELBOURNE')

In [51]:
melb_traffic['LGA_SHORT_NM'].unique()

array([nan, 'MELBOURNE'], dtype=object)

In [52]:
melb_traffic.dropna(subset = ['LGA_SHORT_NM'], inplace = True)
melb_traffic

,OBJECTID_1,OBJECTID,TIS_ID,HMGNS_FLOW_ID,HMGNS_LNK_ID,HMGNS_LNK_DESC,LGA_SHORT_NM,RGN_LONG_NM,ROAD_NBR,DECLARED_ROAD_NM,...,TWO_WAY_AADT_TRUCKS,ALLVEH_AMPEAK_AADT,ALLVEH_PMPEAK_AADT,GROWTH_RATE,CI,AM_PEAK_SPEED,OFF_PEAK_SPEED,PM_PEAK_SPEED,YR,LABEL
95,96.0,1288.0,53.0,53.0,3458.0,FOOTSCRAY ROAD btwn WESTERN LINK TOLLWAY Onra...,MELBOURNE,METROPOLITAN NORTH WEST REGION,2120.0,DOCKLANDS HIGHWAY,...,1700.0,NaN,NaN,-0.001,0.001,NaN,NaN,NaN,2020.0,"16,000* (5% 833*) EAST BOUND"
104,105.0,1297.0,615.0,615.0,4654.0,CITY ROAD btwn ALEXANDRA AVENUE & SOUTHBANK B...,MELBOURNE,METROPOLITAN NORTH WEST REGION,2240.0,YARRA BANK HIGHWAY,...,1400.0,NaN,NaN,0.002,0.002,NaN,NaN,NaN,2020.0,"19,000* (4% 754*) EAST BOUND"
134,135.0,1406.0,1092.0,1092.0,1555.0,WESTERN LINK TOLLWAY btwn WESTERN LINK TOLLWA...,MELBOURNE,METROPOLITAN NORTH WEST REGION,2999.0,WESTERN LINK TOLLWAY,...,14000.0,NaN,NaN,0.026,0.003,NaN,NaN,NaN,2020.0,"58,000* (13% 7,400*) SOUTH BOUND"
156,157.0,1311.0,1359.0,1359.0,2886.0,HARKER STREET btwn FLEMINGTON ROAD & ERROL ST...,MELBOURNE,METROPOLITAN NORTH WEST REGION,5026.0,HARKER STREET,...,720.0,NaN,NaN,0.014,0.003,NaN,NaN,NaN,2020.0,"5,800* (6% 361*) SOUTH WEST BOUND"
160,161.0,1315.0,2266.0,2266.0,2919.0,DYNON ROAD btwn DOCK LINK ROAD & LLOYD STREET,MELBOURNE,METROPOLITAN NORTH WEST REGION,5035.0,DYNON ROAD,...,3300.0,NaN,NaN,0.003,0.005,NaN,NaN,NaN,2020.0,"17,000* (10% 1,800*) EAST BOUND"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14548,14549.0,15159.0,15242.0,15242.0,2474.0,FLINDERS STREET btwn ELIZABETH STREET & ST KI...,MELBOURNE,METROPOLITAN NORTH WEST REGION,9999.0,Not Applicable,...,1200.0,NaN,NaN,-0.014,0.006,NaN,NaN,NaN,2020.0,"9,600* (6% 581*) EAST BOUND"
14549,14550.0,15160.0,15868.0,15868.0,2477.0,ELIZABETH STREET btwn COLLINS STREET & BOURKE...,MELBOURNE,METROPOLITAN NORTH WEST REGION,9999.0,Not Applicable,...,540.0,NaN,NaN,-0.007,0.001,NaN,NaN,NaN,2020.0,"4,500* (6% 270*) SOUTH BOUND"
14570,14571.0,33578.0,6447.0,6447.0,8561.0,POWER STREET btwn WEST GATE FREEWAY & SOUTHER...,MELBOURNE,METROPOLITAN NORTH WEST REGION,8040.0,7824F 2240F,...,760.0,NaN,NaN,0.022,0.004,NaN,NaN,NaN,2020.0,"15,000* (5% 764*) SOUTH EAST BOUND"
14589,14590.0,15446.0,13346.0,13346.0,7409.0,BOURKE STREET btwn SWANSTON STREET WALK & RUS...,MELBOURNE,METROPOLITAN NORTH WEST REGION,9999.0,Not Applicable,...,NaN,NaN,NaN,-0.012,0.003,NaN,NaN,NaN,2020.0,"3,700* (N/A) WEST BOUND"


In [53]:
list(melb_traffic.columns)

['OBJECTID_1',
 'OBJECTID',
 'TIS_ID',
 'HMGNS_FLOW_ID',
 'HMGNS_LNK_ID',
 'HMGNS_LNK_DESC',
 'LGA_SHORT_NM',
 'RGN_LONG_NM',
 'ROAD_NBR',
 'DECLARED_ROAD_NM',
 'LOCAL_ROAD_NM',
 'ALT_ROAD_NM',
 'RMA_CLSFCN_CD',
 'RMA_CLSFCN_GROUP',
 'RMA_DESC',
 'SRNS',
 'SRNS_CD',
 'SRNS_DESC',
 'RMC',
 'FLOW',
 'HMGNS_FLOW_LENGTH',
 'ALLVEHS_MMW',
 'ALLVEH_CALC',
 'ALLVEHS_AADT',
 'BUSES_MMW',
 'BUSES_AADT',
 'TRAMS_MMW',
 'TRAMS_AADT',
 'MOTORCYCLES_MMW',
 'MOTORCYCLES_AADT',
 'MOTORCYCLE_CALC',
 'TRUCKS_MMW',
 'TRUCKS_AADT',
 'TRUCK_CALC',
 'PER_TRUCKS_AADT',
 'EHV',
 'PER_HV',
 'PER_LV',
 'ESA',
 'TWO_WAY_AADT',
 'TWO_WAY_AADT_TRUCKS',
 'ALLVEH_AMPEAK_AADT',
 'ALLVEH_PMPEAK_AADT',
 'GROWTH_RATE',
 'CI',
 'AM_PEAK_SPEED',
 'OFF_PEAK_SPEED',
 'PM_PEAK_SPEED',
 'YR',
 'LABEL']

In [54]:
melb_traffic.iloc[0]#['HMGNS_LNK_DESC']

OBJECTID_1                                                            96
OBJECTID                                                            1288
TIS_ID                                                                53
HMGNS_FLOW_ID                                                         53
HMGNS_LNK_ID                                                        3458
HMGNS_LNK_DESC          FOOTSCRAY ROAD btwn WESTERN LINK TOLLWAY Onra...
LGA_SHORT_NM                                                   MELBOURNE
RGN_LONG_NM                               METROPOLITAN NORTH WEST REGION
ROAD_NBR                                                            2120
DECLARED_ROAD_NM                                       DOCKLANDS HIGHWAY
LOCAL_ROAD_NM                                             FOOTSCRAY ROAD
ALT_ROAD_NM                                               Not Applicable
RMA_CLSFCN_CD                                                         AH
RMA_CLSFCN_GROUP                                   

### Tram Tracks Dataset 

In [55]:
# https://data.melbourne.vic.gov.au/Transport/Tram-tracks/wqka-kyhz

apptoken = os.environ.get("SODAPY_APPTOKEN") # Anonymous App Token
domain = "data.melbourne.vic.gov.au"
client = Socrata(domain, apptoken)           # Open Dataset Connection
tram_data_unique_identifier = 'au2t-98pn'   

tram_track_data = gpd.GeoDataFrame.from_dict(client.get_all(tram_data_unique_identifier))

In [56]:
#tram_track_data.rename(columns={'the_geom':'geometry'}, inplace=True)
print(tram_track_data.shape)
print(tram_track_data.dtypes)

tram_track_data.head()

(645, 3)
the_geom      object
name          object
descriptio    object
dtype: object


,the_geom,name,descriptio
0,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",kml_1,<center><table><tr><th colspan='2' align='cent...
1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",kml_2,<center><table><tr><th colspan='2' align='cent...
2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",kml_3,<center><table><tr><th colspan='2' align='cent...
3,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",kml_4,<center><table><tr><th colspan='2' align='cent...
4,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",kml_5,<center><table><tr><th colspan='2' align='cent...


In [57]:

# Functions for converting GeoJSON datasets into GeoPandas Formats. 
def get_coords(row):
    """
    Converts the geometry column entries of the datafile row into shapely polygons.
    row: a single row entry for a dataframe  
    Code Snippit from: Ideal Green Roof Locations: Melbourne 
    """
    row["geometry"] = Polygon(row['geometry']['coordinates'][0][0])

    return row


def convert_dataset(dataframe):
    """
    Converts the downloaded GeoJSON dataset into a usable Geopandas format
    using the 4326 coordinate system.
    """
    # Rename the column to be recognisable to geopandas
    new_dataframe = dataframe.rename(columns={"the_geom" : "geometry"})

    # Apply the get_coords method to each row
    new_dataframe.apply(get_coords, axis=1)

    return gpd.GeoDataFrame(new_dataframe, geometry='geometry', crs=4326)

In [58]:
tram_track_data_gpd = convert_dataset(tram_track_data)

In [59]:

# Save the layer view as a map layer
tram_tracks_map = tram_track_data_gpd.explore(tiles='CartoDB positron', name = 'Tram Tracks', color='green')
folium.LayerControl().add_to(tram_tracks_map)
tram_tracks_map

### Bike Route Data

In [614]:
apptoken = os.environ.get("SODAPY_APPTOKEN") # Anonymous App Token
domain = "data.melbourne.vic.gov.au"
client = Socrata(domain, apptoken)           # Open Dataset Connection
bike_data_unique_identifier = 'hmuz-nz6m'   

# bike_route_data = gpd.GeoDataFrame.from_dict(client.get_all(bike_data_unique_identifier))

bike_geo_data_url = 'https://'+ domain +'/api/geospatial/'+ bike_data_unique_identifier +'?method=export&format=GeoJSON'
with urlopen(bike_geo_data_url) as result:
    bike_route_data = json.load(result)

bike_route_data.keys()

dict_keys(['type', 'features'])

In [615]:
bike_route_data['features'][0]['properties'].keys()

dict_keys(['name', 'direction', 'info', 'status', 'notes', 'type'])

In [62]:
#bike_route_data['the_geom'] = [Polygon(mapping(x)['coordinates']) for x in bike_route_data['the_geom']]


In [617]:
# Create Base Layer
m = folium.Map(
    location=[-37.81368709240999, 144.95738102347036],
    tiles='Cartodb Positron',
    zoom_start=13,
    min_zoom=10)
folium.GeoJson(
    bike_route_data,
    name='Bike Routes',
    tooltip=folium.features.GeoJsonTooltip(fields=['name','direction','type','notes'])
    ).add_to(m)
folium.LayerControl().add_to(m)

m

#### VicRoads Transportation Accidents Dataset

In [63]:
# This url is for a zip file which contains multiple csv files
crash_url = 'https://vicroadsopendatastorehouse.vicroads.vic.gov.au/opendata/Road_Safety/ACCIDENT.zip'

In [64]:
crash_request = requests.get(crash_url)

Read the zip file and view filename contents

In [65]:
crash_files = zf.ZipFile(BytesIO(crash_request.content))
print(crash_files.namelist())

['ACCIDENT.csv', 'ACCIDENT_CHAINAGE.csv', 'ACCIDENT_EVENT.csv', 'ACCIDENT_LOCATION.csv', 'ATMOSPHERIC_COND.csv', 'NODE.csv', 'NODE_ID_COMPLEX_INT_ID.csv', 'PERSON.csv', 'ROAD_SURFACE_COND.csv', 'Statistic Checks.csv', 'SUBDCA.csv', 'VEHICLE.csv']


Save the 'ACCIDENT.csv' file to a temp folder and load into a dataframe

In [66]:
accident = crash_files.extract('ACCIDENT.csv', 'temp')
accident = pd.read_csv(accident, low_memory=False)
accident.head()

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,6,0,1,0,5,1,1,Cross intersection,3,60
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,4,0,1,0,3,1,2,T intersection,3,70
2,T20060000022,14/01/2006,12:10:00,7,Fall from or in moving vehicle,7,Saturday,190,FELL IN/FROM VEHICLE,MEL,...,2,1,0,0,1,1,5,Not at intersection,2,100
3,T20060000023,14/01/2006,11:49:00,1,Collision with vehicle,7,Saturday,130,REAR END(VEHICLES IN SAME LANE),MEL,...,2,1,0,0,1,1,2,T intersection,2,80
4,T20060000026,14/01/2006,10:45:00,1,Collision with vehicle,7,Saturday,121,RIGHT THROUGH,MEL,...,3,0,3,0,0,1,5,Not at intersection,3,50


### Super Tuesday / Super Sunday Bike useage Data

#### Tuesday - Weekday

In [67]:
# Getting the data

data_file_tuesday = 'uyp8-7ii8'
# super_tuesday = gpd.GeoDataFrame.from_dict(client.get_all(data_file_tuesday))
super_tuesday = pd.DataFrame.from_dict(client.get_all(data_file_tuesday))
print(super_tuesday.shape)
super_tuesday.head(3)



(352, 66)


,state,electorate,site_id,latitude,longitude,legs,description,layout_1,layout_1_enter,layout_2,...,female,male,_7_00_am,_7_15_am,_7_30_am,_7_45_am,_8_00_am,_8_15_am,_8_30_am,_8_45_am
0,VIC,Melbourne,4399,-37.787979,144.959,2,"Royal Pde/shared path [N], Royal Pde/shared pa...",5,185,186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VIC,Melbourne,4405,-37.793993,144.941956,3,"Melrose St [N], Melrose St [S], Mark St [W]",8,188,188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VIC,Melbourne,4406,-37.794117,144.927689,4,"McCracken St [N], Macaulay Rd [E], Kensington ...",0,180,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
list(super_tuesday.columns)[:10]

['state',
 'electorate',
 'site_id',
 'latitude',
 'longitude',
 'legs',
 'description',
 'layout_1',
 'layout_1_enter',
 'layout_2']

In [69]:
super_tuesday_melb = super_tuesday[['state','electorate','site_id','latitude','longitude','legs','description','total','year', '_7_00_am', '_7_15_am','_7_30_am','_7_45_am','_8_00_am','_8_15_am','_8_30_am','_8_45_am']]

In [70]:
super_tuesday_melb.head(2)

,state,electorate,site_id,latitude,longitude,legs,description,total,year,_7_00_am,_7_15_am,_7_30_am,_7_45_am,_8_00_am,_8_15_am,_8_30_am,_8_45_am
0,VIC,Melbourne,4399,-37.787979,144.959,2,"Royal Pde/shared path [N], Royal Pde/shared pa...",1017,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VIC,Melbourne,4405,-37.793993,144.941956,3,"Melrose St [N], Melrose St [S], Mark St [W]",20,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# Covert year column to int
years = [int(x) for x in super_tuesday_melb['year'].unique()]

#get latest poll data
super_tuesday_melb = super_tuesday_melb.loc[super_tuesday_melb['year'] == str(max(years))]
super_tuesday_melb.head()

,state,electorate,site_id,latitude,longitude,legs,description,total,year,_7_00_am,_7_15_am,_7_30_am,_7_45_am,_8_00_am,_8_15_am,_8_30_am,_8_45_am
306,VIC,Melbourne,6554,-37.825963,144.960053,4,"Queens Bridge St [N], City Rd [NE], Morray St ...",249,2017,6,26,37,23,46,35,33,43
307,VIC,Melbourne,4399,-37.787979,144.959,2,"Royal Pde/shared path [N], Royal Pde/shared pa...",1425,2017,72,78,126,160,204,240,258,287
308,VIC,Melbourne,4405,-37.793993,144.941956,3,"Melrose St [N], Melrose St [S], Mark St [W]",42,2017,2,5,6,4,4,2,8,11
309,VIC,Melbourne,4406,-37.794117,144.927689,4,"McCracken St [N], Macaulay Rd [E], Kensington ...",114,2017,9,8,15,17,14,16,17,18
310,VIC,Melbourne,4407,-37.79454,144.930987,4,"Macaulay Rd - east end [E], Eastwood St [S], M...",247,2017,14,28,29,31,33,39,39,34


In [72]:
super_tuesday_melb.dtypes

state          object
electorate     object
site_id        object
latitude       object
longitude      object
legs           object
description    object
total          object
year           object
_7_00_am       object
_7_15_am       object
_7_30_am       object
_7_45_am       object
_8_00_am       object
_8_15_am       object
_8_30_am       object
_8_45_am       object
dtype: object

In [73]:
tuesday_int = ['year', 'total', 'legs','_7_00_am', '_7_15_am','_7_30_am','_7_45_am','_8_00_am','_8_15_am','_8_30_am','_8_45_am']
for col in tuesday_int:
    super_tuesday_melb[col] = super_tuesday_melb[col].astype(int)
    
tuesday_float = ['latitude','longitude']
for col in tuesday_float:
    super_tuesday_melb[col] = super_tuesday_melb[col].astype(float)
    
tuesday_string = ['state','electorate','site_id', 'description']
for col in tuesday_string:
    super_tuesday_melb[col] = super_tuesday_melb[col].astype(str)

### Add geometry

In [74]:
tuesday_geometry = [Point(xy) for xy in zip(super_tuesday_melb['latitude'], super_tuesday_melb['longitude'])]
tuesday_geometry[:4]

In [75]:
super_tuesday_melb_geo = gpd.GeoDataFrame(super_tuesday_melb, crs = 4326, geometry = tuesday_geometry)
super_tuesday_melb_geo.head()

,state,electorate,site_id,latitude,longitude,legs,description,total,year,_7_00_am,_7_15_am,_7_30_am,_7_45_am,_8_00_am,_8_15_am,_8_30_am,_8_45_am,geometry
306,VIC,Melbourne,6554,-37.825963,144.960053,4,"Queens Bridge St [N], City Rd [NE], Morray St ...",249,2017,6,26,37,23,46,35,33,43,POINT (-37.82596 144.96005)
307,VIC,Melbourne,4399,-37.787979,144.959000,2,"Royal Pde/shared path [N], Royal Pde/shared pa...",1425,2017,72,78,126,160,204,240,258,287,POINT (-37.78798 144.95900)
308,VIC,Melbourne,4405,-37.793993,144.941956,3,"Melrose St [N], Melrose St [S], Mark St [W]",42,2017,2,5,6,4,4,2,8,11,POINT (-37.79399 144.94196)
309,VIC,Melbourne,4406,-37.794117,144.927689,4,"McCracken St [N], Macaulay Rd [E], Kensington ...",114,2017,9,8,15,17,14,16,17,18,POINT (-37.79412 144.92769)
310,VIC,Melbourne,4407,-37.794540,144.930987,4,"Macaulay Rd - east end [E], Eastwood St [S], M...",247,2017,14,28,29,31,33,39,39,34,POINT (-37.79454 144.93099)


In [76]:
com = city_suburbs.explore(name = 'Melbourne Cities')
com

In [84]:
tuesday_coords = [(lat, lon) for lat, lon in zip(super_tuesday_melb['latitude'], super_tuesday_melb['longitude'])]
tuesday_coords[:10]

[(-37.825963, 144.960053),
 (-37.787979, 144.959),
 (-37.793993, 144.941956),
 (-37.794117, 144.927689),
 (-37.79454, 144.930987),
 (-37.795096, 144.935852),
 (-37.795421, 144.93885),
 (-37.796627, 144.951172),
 (-37.799747, 144.957474),
 (-37.800289, 144.943954)]

In [124]:
m = folium.Map(location = (-37.81368709240999, 144.95738102347036), zoom_start=13,min_zoom=10)

In [125]:
plugins.HeatMap(tuesday_coords, radius = 20).add_to(m)

In [126]:
m


In [106]:
import plotly.graph_objects as go

In [122]:
lat= super_tuesday_melb['latitude']
lon = super_tuesday_melb['longitude']
tues_geo = pd.DataFrame({'Lat':lat, 'Long':lon, 'Weight':super_tuesday_melb['total']})



map_tuesday = go.Figure(go.Densitymapbox(lat= tues_geo.Lat, lon = tues_geo.Long, 
                                         z = tues_geo.Weight, radius = 50,
                                         colorscale=[[0.0, 'blue',],[0.3,'lime'],[0.5,'yellow'],[0.7,'orange'],[1.0, 'red']],
                                         # custome colorscale
                                 zmin=0.0,
                                 zmax=1.0
                                ))

map_tuesday.update_layout(mapbox_style="carto-positron",
                  mapbox_center_lon=144.95738102347036,
                  mapbox_center_lat=-37.81368709240999,
                  mapbox_zoom=12)

map_tuesday.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
map_tuesday.show()

In [77]:
# com = city_suburbs.explore(name = 'Melbourne Cities')

# city_suburbs.join(population_layers[2022].set_index(["suburb"]), on=["suburb"]).explore(m = com, column = 'pop_2022', name = 'Population (thousands) 2022')
# city_suburbs.join(population_layers[2030].set_index(["suburb"]), on=["suburb"]).explore(m = com, column = 'pop_2030', name = 'Population (thousands) 2030')
# city_suburbs.join(population_layers[2040].set_index(["suburb"]), on=["suburb"]).explore(m = com, column = 'pop_2040', name = 'Population (thousands) 2040')

# folium.TileLayer(control=True).add_to(com)  # use folium to add alternative tiles
# folium.LayerControl().add_to(com)
# com

#### Sunday - Weekend

In [78]:
# Getting the data

data_file_sunday = 'v9mu-dvfu'
# super_sunday = gpd.GeoDataFrame.from_dict(client.get_all(data_file_sunday))
super_sunday = pd.DataFrame.from_dict(client.get_all(data_file_sunday))
print(super_sunday.shape)
super_sunday.head(3)

(59, 51)


,state,electorate,site_id,latitude,longitude,legs,description,layout_1,layout_1_enter,layout_2,...,runner,dog,other,total,year,location,_9_00_00,_10_00_00,_11_00_00,_12_00_00
0,VIC,Melbourne,5666,-37.825893,145.137027,4,"Middleborough Rd [N], Gardiners Crk Trail [E],...",7,187,89,...,4,6,3,77,2011,"{'type': 'Point', 'coordinates': [145.137027, ...",NaN,NaN,NaN,NaN
1,VIC,Melbourne,5608,-37.777661,144.896519,4,"Maribyrnong River Trail #5 [E], Maribyrnong Ri...",83,263,178,...,30,21,1,489,2011,"{'type': 'Point', 'coordinates': [144.896519, ...",NaN,NaN,NaN,NaN
2,VIC,Melbourne,5609,-37.806401,144.90809,3,"Maribyrnong River Trail [N], Docklands Hwy [E]...",16,196,111,...,10,2,0,279,2011,"{'type': 'Point', 'coordinates': [144.90809, -...",NaN,NaN,NaN,NaN


In [79]:
list(super_sunday.columns)

['state',
 'electorate',
 'site_id',
 'latitude',
 'longitude',
 'legs',
 'description',
 'layout_1',
 'layout_1_enter',
 'layout_2',
 'layout_2_enter',
 'layout_3',
 'layout_3_enter',
 'layout_4',
 'layout_4_enter',
 'leg1_2',
 'leg1_3',
 'leg1_4',
 'leg2_1',
 'leg2_3',
 'leg2_4',
 'leg3_1',
 'leg3_2',
 'leg3_4',
 'leg4_1',
 'leg4_2',
 'leg4_3',
 'leg1_enter',
 'leg1_exit',
 'leg1_total',
 'leg2_enter',
 'leg2_exit',
 'leg2_total',
 'leg3_enter',
 'leg3_exit',
 'leg3_total',
 'leg4_enter',
 'leg4_exit',
 'leg4_total',
 'bicycle',
 'walker',
 'runner',
 'dog',
 'other',
 'total',
 'year',
 'location',
 '_9_00_00',
 '_10_00_00',
 '_11_00_00',
 '_12_00_00']

In [80]:
super_sunday_melb = super_sunday[['state','electorate','site_id','latitude','longitude','legs','description','total','year', '_9_00_00', '_10_00_00','_11_00_00','_12_00_00']]

In [81]:
years = [int(x) for x in super_sunday_melb['year'].unique()]

super_sunday_melb = super_sunday_melb.loc[super_sunday_melb['year'] == str(max(years))]
super_sunday_melb.head()

,state,electorate,site_id,latitude,longitude,legs,description,total,year,_9_00_00,_10_00_00,_11_00_00,_12_00_00
50,VIC,Melbourne,5609,-37.806401,144.90809,3,"Maribyrnong River Trail [N], Docklands Hwy [E]...",596,2017,157,166,175,98
51,VIC,Melbourne,5610,-37.809895,144.935722,4,"Moonee Ponds Crk Trail [N], Footscray Rd [E], ...",190,2017,42,60,52,36
52,VIC,Melbourne,5611,-37.823699,144.956026,4,"Southbank Prom [NE], Clarendon St [SE], Conven...",9170,2017,1320,1995,2686,3169
53,VIC,Melbourne,5612,-37.821176,144.97603,3,"Batman Ave path [N], Main Yarra Trail [SE], Ma...",1144,2017,140,162,300,542
54,VIC,Melbourne,5613,-37.828185,144.984906,4,"Morrell St Bridge [N], Capital City Trail [SE]...",1922,2017,518,528,392,484


In [82]:
#Convert columns

sunday_int = ['year', 'total', 'legs','_9_00_00', '_10_00_00','_11_00_00','_12_00_00']
for col in sunday_int:
    super_sunday_melb[col] = super_sunday_melb[col].astype(int)
    
sunday_float = ['latitude','longitude']
for col in sunday_float:
    super_sunday_melb[col] = super_sunday_melb[col].astype(float)
    
sunday_string = ['state','electorate','site_id', 'description']
for col in sunday_string:
    super_sunday_melb[col] = super_sunday_melb[col].astype(str)